In [2]:
!kubecolor -n postgres get all

NAME                        READY   STATUS    RESTARTS   AGE
pod/postgres-postgresql-0   1/1     Running   0          64m

NAME                             TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/postgres-postgresql      ClusterIP   10.97.123.12   <none>        5432/TCP   25h
service/postgres-postgresql-hl   ClusterIP   None           <none>        5432/TCP   25h

NAME                                   READY   AGE
statefulset.apps/postgres-postgresql   1/1     25h


In [4]:
!kubecolor -n postgres get services

NAME                     TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
postgres-postgresql      ClusterIP   10.97.123.12   <none>        5432/TCP   25h
postgres-postgresql-hl   ClusterIP   None           <none>        5432/TCP   25h


In [6]:
!kubectl -n postgres get services postgres-postgresql -o yaml | yq

apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: postgres
    meta.helm.sh/release-namespace: postgres
  creationTimestamp: "2024-02-02T10:41:58Z"
  labels:
    app.kubernetes.io/component: primary
    app.kubernetes.io/instance: postgres
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: postgresql
    app.kubernetes.io/version: 16.1.0
    helm.sh/chart: postgresql-13.2.30
  name: postgres-postgresql
  namespace: postgres
  resourceVersion: "26761"
  uid: c7683b9e-5ece-4700-ba5c-698c58202b8c
spec:
  clusterIP: 10.97.123.12
  clusterIPs:
    - 10.97.123.12
  internalTrafficPolicy: Cluster
  ipFamilies:
    - IPv4
  ipFamilyPolicy: SingleStack
  ports:
    - name: tcp-postgresql
      port: 5432
      protocol: TCP
      targetPort: tcp-postgresql
  selector:
    app.kubernetes.io/component: primary
    app.kubernetes.io/instance: postgres
    app.kubernetes.io/name: postgresql
  sessionAffinity: None
  type: ClusterIP
status:
  loadBa

In [8]:
!nmap -p 5432 postgres-postgresql.postgres.svc

Starting Nmap 7.80 ( https://nmap.org ) at 2024-02-03 08:50 -03
Nmap scan report for postgres-postgresql.postgres.svc (10.97.123.12)
Host is up (0.0012s latency).
rDNS record for 10.97.123.12: postgres-postgresql.postgres.svc.cluster2.xpt

PORT     STATE SERVICE
5432/tcp open  postgresql

Nmap done: 1 IP address (1 host up) scanned in 0.12 seconds


In [2]:
! POSTGRES_HOST=postgres-postgresql.postgres.svc.cluster2.xpt && \
  export POSTGRES_PASSWORD=$(kubectl get secret --namespace postgres postgres-postgresql -o jsonpath="{.data.postgres-password}" | base64 -d) && \
  echo $POSTGRES_HOST, $POSTGRES_PASSWORD

postgres-postgresql.postgres.svc.cluster2.xpt, ktP9ZEYVrH


In [7]:
! POSTGRES_HOST=postgres-postgresql.postgres.svc.cluster2.xpt && \
  export POSTGRES_PASSWORD=$(kubectl get secret --namespace postgres postgres-postgresql -o jsonpath="{.data.postgres-password}" | base64 -d) && \
  kubectl run postgres-postgresql-client --rm --tty -i --restart='Never' \
                                       --namespace postgres --image docker.io/bitnami/postgresql:15.1.0-debian-11-r20 \
                                       --env="PGPASSWORD=$POSTGRES_PASSWORD" \
                                       --command -- ls /
 

bin	 dev				etc   lib64  opt   run	 sys  var
bitnami  docker-entrypoint-initdb.d	home  media  proc  sbin  tmp
boot	 docker-entrypoint-preinitdb.d	lib   mnt    root  srv	 usr
pod "postgres-postgresql-client" deleted
